In [1]:
import os
if os.name=='nt':
    mingw_path = 'C:\\Program Files\\mingw-w64\\x86_64-8.1.0-posix-seh-rt_v6-rev0\\mingw64\\bin'
    os.environ['PATH'] = mingw_path + ';' + os.environ['PATH']

import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.metrics import matthews_corrcoef, roc_auc_score
from sklearn.cross_validation import cross_val_score, StratifiedKFold
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from sys import getsizeof
import time
import tqdm 
from matplotlib.backends.backend_pdf import PdfPages
import gc


%matplotlib inline

C:\Users\home\Anaconda3\envs\kaggle\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
import pickle

def save_pickle(x, filename):
    with open(filename, 'wb') as handle:
        pickle.dump(x, handle, protocol=pickle.HIGHEST_PROTOCOL)

def read_pickle(filename):
    with open(filename, 'rb') as handle:
        x = pickle.load(handle)
    return x

In [3]:
feats = list(pd.read_csv('important_numeric_features.csv', index_col=0, header=None).values.ravel())
feats.insert(0, 'Id')

In [4]:
train = pd.read_csv('data/train_numeric.csv.zip', index_col=0, usecols=feats, nrows=None)
test = pd.read_csv('data/test_numeric.csv.zip', index_col=0, usecols=feats, nrows=None)
response = pd.read_csv('data/train_numeric.csv.zip', index_col=0, usecols=[0, 969], nrows=None)
n_train = len(train)

C:\Users\home\Anaconda3\envs\kaggle\lib\site-packages\numpy\lib\arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


### All numeric values have only three numbers to the right of the decimal dot
I.e., all the numeric numbers are of the form xx.yyy

In [41]:
# Check if there is any feature has higher resolutions
a = train.iloc[0].fillna(0)*1000
b = a - a.round(0)
b.max() - b.min()

1.4210854715202004e-13

Check if there all the features have only three numbers in the fractional part.
The result below shows that the residual is really small

In [42]:
def rounding(x):
    a = x.fillna(0)*1000
    b = a - a.round(0)
    return b.max() - b.min()

train.apply(lambda x: rounding(x)).max()

2.2737367544323206e-13

But we cannot convert `NaN` to `int`, so I will not continue with this idea.

### Convert dataframes to numpy arrays

In [54]:
x_train_original = train.values
x_test_original = test.values
print(x_train_original.shape)
print(x_test_original.shape)

(1183747, 149)
(1183748, 149)


### Load engineered features

In [55]:
x_train_numeric_feat_0 = read_pickle('x_train_numeric_feat_0.pickle')
x_test_numeric_feat_0 = read_pickle('x_test_numeric_feat_0.pickle')

In [57]:
x_train_numeric_feat_1 = np.concatenate([x_train_original, x_train_numeric_feat_0], axis=1)
x_test_numeric_feat_1 = np.concatenate([x_test_original, x_test_numeric_feat_0], axis=1)

In [58]:
save_pickle(x_train_numeric_feat_1, 'x_train_numeric_feat_1.pickle')
save_pickle(x_test_numeric_feat_1, 'x_test_numeric_feat_1.pickle')